In [108]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Time, Boolean
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import pandas as pd

# Define your SQLAlchemy base
Base = declarative_base()
engine = create_engine("sqlite:///demo.db")


class GpBase(Base):
    __abstract__ = True

    @classmethod
    def from_dataframe(cls, df):
        for _, row in df.iterrows():
            yield cls._instantiate_from_row(row)

    @classmethod
    def _instantiate_from_row(cls, row):
        return cls(**row.to_dict())

    def __repr__(self):
        return f"{self.__class__.__name__}({self.to_dict()})"

    # @classmethod
    # def bulk_insert_from_dataframe(cls, df, session):
    #     instances = [cls._instantiate_from_row(row) for _, row in df.iterrows()]
    #     session.bulk_save_objects(instances)
    #     session.commit()


class GP(GpBase):
    __tablename__ = "gp"
    id = Column(Integer, primary_key=True, autoincrement=True)
    GpName = Column(String)
    GpYear = Column(Integer)

    lap = relationship("GPLap", back_populates="gp", passive_deletes=True)

class Driver(GpBase):
    __tablename__ = "driver"
    id = Column(Integer, primary_key=True, autoincrement=True)
    DriverNumber = Column(String)
    Abbreviation = Column(String)
    FirstName = Column(String)
    LastName = Column(String)
    CountryCode = Column(String)
    
    lap = relationship("GPLap", back_populates="driver", passive_deletes=True)


class GPLap(GpBase):
    __tablename__ = "gplap"
    id = Column(Integer, primary_key=True, autoincrement=True)
    LapTime = Column(Time, nullable=False)
    LapNumber = Column(Integer, nullable=False)
    Position = Column(Integer, nullable=False)
    Compound = Column(String, nullable=False)
    Deleted = Column(Boolean, nullable=False)
    GpId = Column(Integer, ForeignKey("gp.id", ondelete="CASCADE"), index=True, nullable=False)
    DriverId = Column(Integer, ForeignKey("driver.id", ondelete="CASCADE"), index=True, nullable=False)
    
    gp = relationship("GP", back_populates="lap")
    driver = relationship("Driver", back_populates="lap")


tables = [GP, Driver, GPLap]
for table in tables:
    table.__table__.create(bind=engine, checkfirst=True)

In [109]:
# class MyTest(Base):
#     __tablename__ = "mytest"

#     id = Column(Integer, primary_key=True)
#     name = Column(String)


# engine = create_engine("sqlite:///example.db")
# Base.metadata.create_all(engine)
# Session = sessionmaker(bind=engine)
# session = Session()

# # Create an instance of MyTest
# data = {"name": "test"}
# table = MyTest(**data)

# # Add the instance to the session and commit
# session.add(table)
# session.commit()
# print(table)


In [110]:
import fastf1 as ff1
import os

year, gp_id = 2024, 8

race = ff1.get_session(year, gp_id, "R")
race.load(weather=False)

def save_retrieved_data(data, data_type, year, gp_id):
    path = f"retrieved_data/{year}/{gp_id}/"
    os.makedirs(path, exist_ok=True)
    data.to_csv(path + data_type + ".csv", index=False)
    
save_retrieved_data(data=race.laps, data_type="laps", year=year, gp_id=gp_id)

core           INFO 	Loading data for Monaco Grand Prix - Race [v3.3.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '81', '55', '4', '63', '1', '44', '22', '23', '10', '14', '3', '77', '18', '2', '24', '31', '11', '27', '20']


In [111]:
def extract_driver_info(data):
    return data[["Driver", "DriverNumber"]]

In [112]:
for lap in race.laps.iterlaps():
    print(lap["Driver"])

TypeError: tuple indices must be integers or slices, not str

In [113]:
gp = []
drivers = []

def get_drivers_cleaned_data(
    race, keys=["DriverNumber", "Abbreviation", "FirstName", "LastName", "CountryCode"]
):
    for driver in race.drivers:
        data = race.get_driver(driver)
        yield {key: data[key] for key in keys}

def get_laps_cleaned_data(
    race, keys=["LapTime", "LapNumber", "Driver", "Position", "Compound", "Deleted"]
):
    data = race.laps[keys]
    for _, lap in data.iterlaps():
        yield lap


drivers_df = pd.DataFrame(get_drivers_cleaned_data(race))
laps_df = pd.DataFrame(get_laps_cleaned_data(race))

In [114]:
class F1DataHolder:
    table_order = []
    
    def __init__(self, data):
        self.data = data

    @classmethod
    def load(cls, data):
        for table in cls.table_order:
            table.load(data)
            
class GpLapsData(F1DataHolder):
    table_order = [GP, Driver, GPLap]
    

In [115]:
# get the data
laps_df
drivers_df

# create gp Name
gp = GP(GpName="Monaco", GpYear=2024)
# create drivers
drivers = list(Driver.from_dataframe(drivers_df))
# create laps
laps = list(GPLap.from_dataframe(laps_df))
# bind laps to drivers
# bind laps to gp

# load gp
# load drivers
# load laps

# get the second gp
# create gp Name
# create drivers
# create laps

# create gp
# create drivers
# create laps

# bind laps to drivers
# bind laps to gp

# check if gp exists
# load gp
# check if drivers exists
# load drivers
# check if laps exists
# load laps


TypeError: 'Driver' is an invalid keyword argument for GPLap

In [107]:
laps_df

,LapTime,LapNumber,Driver,Position,Compound,Deleted
0,0 days 00:40:56.330000,1.0,LEC,1.0,HARD,False
1,0 days 00:01:24.624000,2.0,LEC,1.0,HARD,False
2,0 days 00:01:22.625000,3.0,LEC,1.0,HARD,False
3,0 days 00:01:21.795000,4.0,LEC,1.0,HARD,False
4,0 days 00:01:21.145000,5.0,LEC,1.0,HARD,False
...,...,...,...,...,...,...
1216,0 days 00:01:17.173000,75.0,ZHO,16.0,SOFT,False
1217,NaT,1.0,OCO,3.0,HARD,False
1218,NaT,1.0,PER,2.0,HARD,False
1219,NaT,1.0,HUL,4.0,MEDIUM,False
